<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2d5e4dd0fdda68e2e4d38222fa7560a503c136558c422e707d09b125a2cef414
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-31 11:15:51
-------------------
qualified stocks: 88
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.37 K
Current:  1.43 C
-------------------
Today PnL: 1.00 L (0.71%)
Current PnL: -26.24 L (-16.73%)
CY Booked + Current PnL: -11.95 L (-7.62%)
-------------------
Total profit:  1.47 L
Total loss:  -27.70 L
-------------------
Total Booked + Current PnL: 15.03 L (11.58%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.01%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 91.62 L (64.15%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.27%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,1.23,-19.41,24.19,0.09,19647.0,-19557.0,81220.0,88.47,41.0,M-SC,3.17,253.0,-1.00,0.57,6.71,OX40N,NTT,DURABLES
53,NESTLEIND,1377.00,0.56,12.95,7.48,21.40,22425.0,34375.0,299801.0,4.80,64.0,X-LC,5.19,12.0,1.53,2.10,20.10,XY25,NTT,FMCG
51,MASFIN,398.61,2.19,-1.61,24.04,22.05,23176.0,-1575.0,96405.0,-15.09,55.0,H-SC,6.57,164.0,-0.07,0.68,40.13,XR,ATH,FINANCE
84,VOLTAS,1530.00,0.53,6.23,12.67,19.69,25807.0,11943.0,203685.0,-4.12,46.0,X-MC,2.13,78.0,0.46,1.43,13.45,XY25,NTT,AC
44,ITC,452.00,0.55,-1.28,12.21,10.78,28968.0,-3069.0,237249.0,-41.15,48.0,X-LC,1.99,5.0,-0.11,1.66,3.89,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,4.70,-30.54,178.75,93.63,125363.0,-30830.0,70133.0,-25.56,48.0,H-SC,15.68,145.0,-0.25,0.49,4.70,SR,ATH,CHEMICALS
49,KPIGREEN,730.66,4.51,-0.32,47.43,46.95,59236.0,-406.0,124891.0,-8.00,68.0,H-SC,9.43,154.0,-0.01,0.87,56.46,MH,ATH,POWER
10,BAJAJHLDNG,14451.52,2.87,1.65,27.92,30.03,37849.0,2194.0,135564.0,-1.33,50.0,X-LC,12.17,35.0,0.06,0.95,10.29,X40,ATH,FINANCE
66,SHALBY,327.00,2.27,-23.10,62.69,25.10,87323.0,-41850.0,139293.0,923.92,47.0,M-SC,9.16,250.0,-0.48,0.98,11.63,XY24,NTT,HEALTHCARE
59,RECLTD,446.00,2.26,-2.72,24.11,20.74,47651.0,-5528.0,197642.0,28.81,59.0,M-MC,8.41,189.0,-0.12,1.38,7.61,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-2.05,-37.94,117.98,35.27,83133.0,-43085.0,70464.0,-51.71,54.0,L-SC,7.02,270.0,-0.52,0.49,72.46,XR,NTT,HOTELS
39,INDIAMART,4810.62,-1.46,-4.86,117.28,106.72,137619.0,-5994.0,117342.0,-52.92,41.0,H-SC,5.87,139.0,-0.04,0.82,17.38,AR,ATH,MISC
57,RAJESHEXPO,518.00,-1.07,-63.30,172.63,0.07,86919.0,-86827.0,50350.0,1745.96,45.0,L-SC,4.43,268.0,-1.00,0.35,25.22,OX40N,NTT,JEWELLERY
47,JSWINFRA,342.00,-0.85,-3.21,20.30,16.43,43573.0,-7128.0,214646.0,-17.97,57.0,X-MC,7.64,66.0,-0.16,1.50,26.68,X40N,NTT,REALTY
75,TCS,4389.97,-0.85,-11.49,36.37,20.71,121762.0,-43451.0,334786.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.86,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.21,-0.86,83.25,81.67,77423.0,-809.0,93001.0,8377.78,66.0,L-SC,16.01,271.0,-0.01,0.65,80.10,XR,NTT,CERAMICS
49,KPIGREEN,730.66,4.51,-0.32,47.43,46.95,59236.0,-406.0,124891.0,-8.00,68.0,H-SC,9.43,154.0,-0.01,0.87,56.46,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,1.25,-6.15,121.55,107.91,168417.0,-9086.0,138558.0,-23.79,42.0,M-SC,11.64,216.0,-0.05,0.97,3.31,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,1.70,-18.91,23.34,0.02,33040.0,-33001.0,141558.0,115.96,39.0,M-SC,8.36,234.0,-1.00,0.99,23.13,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,1.23,-19.41,24.19,0.09,19647.0,-19557.0,81220.0,88.47,41.0,M-SC,3.17,253.0,-1.00,0.57,6.71,OX40N,NTT,DURABLES
48,KANSAINER,340.00,0.80,-21.32,44.23,13.47,93841.0,-57501.0,212166.0,-67.73,55.0,H-SC,6.49,158.0,-0.61,1.49,7.88,XY24,NTT,PAINTS
68,SIS,528.00,0.00,-23.38,58.27,21.26,49569.0,-25964.0,85068.0,2007.77,52.0,H-SC,4.48,166.0,-0.52,0.60,14.94,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-0.29,7.83,59.76,72.27,109187.0,13261.0,182709.0,-7.44,56.0,M-LC,5.22,99.0,0.12,1.28,14.48,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.00,-0.29,7.83,59.76,72.27,109187.0,13261.0,182709.0,-7.44,56.0,M-LC,5.22,99.0,0.12,1.28,14.48,XR,NTT,IT
6,ASIANTILES,137.00,0.21,-0.86,83.25,81.67,77423.0,-809.0,93001.0,8377.78,66.0,L-SC,16.01,271.0,-0.01,0.65,80.10,XR,NTT,CERAMICS
49,KPIGREEN,730.66,4.51,-0.32,47.43,46.95,59236.0,-406.0,124891.0,-8.00,68.0,H-SC,9.43,154.0,-0.01,0.87,56.46,MH,ATH,POWER
39,INDIAMART,4810.62,-1.46,-4.86,117.28,106.72,137619.0,-5994.0,117342.0,-52.92,41.0,H-SC,5.87,139.0,-0.04,0.82,17.38,AR,ATH,MISC
26,FINCABLES,1641.55,1.25,-6.15,121.55,107.91,168417.0,-9086.0,138558.0,-23.79,42.0,M-SC,11.64,216.0,-0.05,0.97,3.31,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.17,-19.47,93.06,55.47,182320.0,-47369.0,195917.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.38,X40N,ATH,FINANCE
23,DIXON,18931.72,1.00,-20.86,59.29,26.07,91606.0,-40716.0,154505.0,-61.53,26.0,X-MC,16.14,56.0,-0.44,1.08,1.00,X40N,ATH,IT
35,HONAUT,58357.33,-0.08,-19.03,77.00,43.32,101548.0,-30992.0,131880.0,-30.40,28.0,X-SC,7.22,90.0,-0.31,0.92,1.26,X40N,ATH,ELECTRICAL
28,GLAXO,3466.20,0.75,-3.57,44.30,39.15,85127.0,-7124.0,192160.0,-22.56,31.0,X-MC,10.86,60.0,-0.08,1.35,24.39,X40,ATH,PHARMA
3,ACC,3906.00,-0.16,-27.33,125.87,64.15,217667.0,-65021.0,172930.0,-57.62,33.0,X-SC,5.67,82.0,-0.30,1.21,0.24,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.5,1.07,-17.73,52.56,25.51,80469.0,-32995.0,153100.0,-6.04,40.0,H-LC,0.98,49.0,-0.41,1.07,12.82,AR,ATH,ELECTRICAL
80,UNITDSPR,1644.0,0.27,4.88,15.15,20.77,37203.0,11420.0,245564.0,-4.29,50.0,X-MC,1.53,62.0,0.31,1.72,12.13,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,1.46,-5.01,27.53,21.14,59450.0,-11393.0,215946.0,-11.79,42.0,X-MC,1.63,74.0,-0.19,1.51,19.92,XY24,NTT,PAINTS
21,DABUR,735.0,2.22,-0.94,45.07,43.70,110748.0,-2343.0,245725.0,-9.06,58.0,X-MC,1.99,72.0,-0.02,1.72,13.96,XY24,BTT,FMCG
44,ITC,452.0,0.55,-1.28,12.21,10.78,28968.0,-3069.0,237249.0,-41.15,48.0,X-LC,1.99,5.0,-0.11,1.66,3.89,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.16,-27.33,125.87,64.15,217667.0,-65021.0,172930.0,-57.62,33.0,X-SC,5.67,82.0,-0.30,1.21,0.24,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-0.17,-19.47,93.06,55.47,182320.0,-47369.0,195917.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.38,X40N,ATH,FINANCE
20,COLPAL,3726.84,0.71,-21.48,80.22,41.51,165887.0,-56575.0,206790.0,-6.53,35.0,X-MC,8.39,61.0,-0.34,1.45,0.71,XY25,ATH,FMCG
12,BATAINDIA,2096.00,0.85,-38.97,121.55,35.20,95443.0,-50148.0,78522.0,2.96,37.0,X-SC,17.35,92.0,-0.53,0.55,0.85,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,0.94,-10.88,43.75,28.11,78531.0,-21910.0,179500.0,-35.36,35.0,X-MC,15.20,55.0,-0.28,1.26,0.94,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.42,-29.45,102.47,42.84,46994.0,-19145.0,45861.0,-53.72,45.0,X-SC,31.58,83.0,-0.41,0.32,3.76,XY24,NTT,MISC
60,RELAXO,1176.00,1.12,-47.56,190.44,52.31,144965.0,-69039.0,76121.0,-44.15,46.0,X-SC,7.38,91.0,-0.48,0.53,1.89,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.85,-38.97,121.55,35.20,95443.0,-50148.0,78522.0,2.96,37.0,X-SC,17.35,92.0,-0.53,0.55,0.85,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.83,-3.07,28.00,24.08,34455.0,-3895.0,123055.0,-10.94,42.0,X-SC,7.70,89.0,-0.11,0.86,15.95,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,-0.08,-19.03,77.00,43.32,101548.0,-30992.0,131880.0,-30.40,28.0,X-SC,7.22,90.0,-0.31,0.92,1.26,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-0.85,-11.49,36.37,20.71,121762.0,-43451.0,334786.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.86,X40,ATH,IT
42,INFY,1972.00,-0.38,9.97,22.08,34.24,75973.0,31192.0,344080.0,-14.21,52.0,X-LC,7.31,2.0,0.41,2.41,19.08,X40,NTT,IT
77,TMPV,600.00,0.93,-25.17,64.34,22.98,108761.0,-56857.0,169041.0,-21.86,56.0,X-LC,5.86,3.0,-0.52,1.18,6.32,XY24,NTT,AUTO
83,VBL,671.64,0.51,-1.65,37.94,35.67,117858.0,-5200.0,310642.0,-13.27,60.0,X-LC,2.92,4.0,-0.04,2.18,11.98,X40N,ATH,FMCG
44,ITC,452.00,0.55,-1.28,12.21,10.78,28968.0,-3069.0,237249.0,-41.15,48.0,X-LC,1.99,5.0,-0.11,1.66,3.89,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,1.91,-36.69,109.89,32.89,53718.0,-28324.0,48883.0,-703.98,58.0,L-MC,6.64,259.0,-0.53,0.34,34.62,XR,NTT,BANKS
6,ASIANTILES,137.00,0.21,-0.86,83.25,81.67,77423.0,-809.0,93001.0,8377.78,66.0,L-SC,16.01,271.0,-0.01,0.65,80.10,XR,NTT,CERAMICS
51,MASFIN,398.61,2.19,-1.61,24.04,22.05,23176.0,-1575.0,96405.0,-15.09,55.0,H-SC,6.57,164.0,-0.07,0.68,40.13,XR,ATH,FINANCE
14,BSOFT,831.70,1.49,-22.06,93.46,50.79,102455.0,-31029.0,109624.0,0.12,53.0,H-SC,11.11,151.0,-0.30,0.77,27.64,XR,ATH,IT
49,KPIGREEN,730.66,4.51,-0.32,47.43,46.95,59236.0,-406.0,124891.0,-8.00,68.0,H-SC,9.43,154.0,-0.01,0.87,56.46,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.21,-0.86,83.25,81.67,77423.0,-809.0,93001.0,8377.78,66.0,L-SC,16.01,271.0,-0.01,0.65,80.10,XR,NTT,CERAMICS
49,KPIGREEN,730.66,4.51,-0.32,47.43,46.95,59236.0,-406.0,124891.0,-8.00,68.0,H-SC,9.43,154.0,-0.01,0.87,56.46,MH,ATH,POWER
14,BSOFT,831.70,1.49,-22.06,93.46,50.79,102455.0,-31029.0,109624.0,0.12,53.0,H-SC,11.11,151.0,-0.30,0.77,27.64,XR,ATH,IT
76,TITAGARH,1548.00,2.11,-15.55,72.58,45.74,148438.0,-37661.0,204516.0,12.55,65.0,H-SC,4.84,174.0,-0.25,1.43,33.02,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.00,1.75,10.26,18.57,30.73,40337.0,20209.0,217214.0,-15.53,69.0,X-MC,4.29,75.0,0.50,1.52,24.34,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.96
1,25,44.13
2,50,75.10


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.17
MC    30.16
LC    25.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.48
X40      23.49
X40N     12.60
XY25      9.95
XR        9.63
AR        8.61
OX40N     7.34
SR        0.99
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.72
H-SC    23.58
X-LC    22.24
M-SC    11.26
X-SC     7.84
H-MC     4.72
L-SC     1.49
M-MC     1.38
M-LC     1.28
H-LC     1.07
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.11,-6.82,41.17
IT,12.50,-17.70,76.87
FINANCE,10.47,-15.43,64.43
MISC,7.01,-29.59,82.60
ELECTRICAL,5.82,-12.85,54.18
PAINTS,5.70,-13.43,30.28
INSURANCE,4.69,-1.21,35.85
PHARMA,3.97,-3.84,36.52
AUTO,2.80,-30.73,76.06


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3140068.0,21
AR,1340373.0,10
XR,1286389.0,13
X40,1042495.0,15
X40N,884928.0,9
OX40N,741603.0,10
XY25,387123.0,7
SR,279342.0,2
MH,59236.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3609256.0,25
M-SC,1449476.0,15
X-MC,1439704.0,16
X-LC,890445.0,13
X-SC,797813.0,8
H-MC,405697.0,3
L-SC,247475.0,3
M-LC,109187.0,1
H-LC,80469.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1240606.0      6
           AR         916357.0      5
M-SC       XY24       817788.0      6
H-SC       XR         784178.0      7
X-MC       X40        488093.0      7
           XY24       405215.0      3
X-LC       X40        371388.0      6
X-MC       X40N       354702.0      4
H-SC       OX40N      329537.0      4
M-SC       OX40N      325147.0      5
X-SC       X40N       315683.0      3
           XY24       299116.0      3
H-SC       SR         279342.0      2
H-MC       AR         215756.0      2
X-LC       X40N       214543.0      2
X-MC       XY25       191694.0      2
H-MC       XY24       189941.0      1
X-LC       XY24       187402.0      2
X-SC       X40        183014.0      2
M-SC       XR         178750.0      2
L-SC       XR         160556.0      2
M-SC       AR         127791.0      2
X-LC       XY25       117112.0      3
M-LC       XR         109187.0      1
L-SC       OX40N       86919.0      1
H-LC       AR          80469.0      1
H-SC       MH          59236.0      1
L-MC       XR          53718.0      1
M-MC       XY25        47651.0      1
L-LC       XY25        30666.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
